<a href="https://colab.research.google.com/github/ayuksekdag/pyspark_learning/blob/main/pyspark_schemas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# step 0: Setup external files
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,167 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.laun

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()

import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

print(sc)

from pyspark.sql import SparkSession
sparkSes = SparkSession.builder.getOrCreate()

print(sparkSes)


<SparkContext master=local[*] appName=pyspark-shell>


In [12]:
# step 3 download crimes files
!wget "https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=%3B"
# rename file
!mv "rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=;" reported-crimes.csv

--2022-04-30 08:26:38--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=%3B
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=;’

rows.csv?accessType     [    <=>             ]   1.64G  3.00MB/s    in 9m 30s  

2022-04-30 08:36:09 (2.95 MB/s) - ‘rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=;’ saved [1761826309]



In [13]:
!ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [24]:
from pyspark.sql.functions import to_timestamp, col, lit
 
df = sparkSes.read.options( \
                        delimiter=';' \
                        ,header='True' \
                        ,dateFormat = "yyyy-MM-dd HH:mm:ss a" \
                        ,inferSchema = "True"\
                        ,nullValue = 'null' \
                        ) \
                        .csv('reported-crimes.csv') 
# df = spark.read.csv('reported-crimes.csv',header=True)
df.show(2)

+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true| 924|   

In [25]:
df.dtypes

[('ID', 'int'),
 ('Case Number', 'string'),
 ('Date', 'string'),
 ('Block', 'string'),
 ('IUCR', 'string'),
 ('Primary Type', 'string'),
 ('Description', 'string'),
 ('Location Description', 'string'),
 ('Arrest', 'boolean'),
 ('Domestic', 'boolean'),
 ('Beat', 'int'),
 ('District', 'int'),
 ('Ward', 'int'),
 ('Community Area', 'int'),
 ('FBI Code', 'string'),
 ('X Coordinate', 'int'),
 ('Y Coordinate', 'int'),
 ('Year', 'int'),
 ('Updated On', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double'),
 ('Location', 'string')]

In [26]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [23]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType

df.columns


['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [ ]:

StructType ([
             StructField ('ID', StringType(), True),
             StructField ('Case Number', IntegerType(), True),
and so on ...
,
 '',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']
)

In [90]:
a = df.dtypes
#print(type(a))
#print(a)
for i in range(len(a)):
#  print(a[i][0])
  if a[i][0] == 'Date':
    a[i]=  '(' + a[i][0]  + ',' + 'TimestampType()'  
  elif a[i][1] == 'int':
    a[i]=  "('" + a[i][0] + "','" + "IntegerType()'"
  elif a[i][1] == 'string':
    a[i]=  "('" + a[i][0] + "','" + "StringType()'"
  elif a[i][1] == 'boolean':
    a[i]=  "('" + a[i][0] + "','" + "BooleanType()'"
  elif a[i][1] == 'double':
    a[i]=  "('" + a[i][0] + "','" + "IntegerType()'"

a

#schema = StructType([StructField (x[0],x[1], True) for x in a ])

["('ID','IntegerType()'",
 "('Case Number','StringType()'",
 '(Date,TimestampType()',
 "('Block','StringType()'",
 "('IUCR','StringType()'",
 "('Primary Type','StringType()'",
 "('Description','StringType()'",
 "('Location Description','StringType()'",
 "('Arrest','BooleanType()'",
 "('Domestic','BooleanType()'",
 "('Beat','IntegerType()'",
 "('District','IntegerType()'",
 "('Ward','IntegerType()'",
 "('Community Area','IntegerType()'",
 "('FBI Code','StringType()'",
 "('X Coordinate','IntegerType()'",
 "('Y Coordinate','IntegerType()'",
 "('Year','IntegerType()'",
 "('Updated On','StringType()'",
 "('Latitude','IntegerType()'",
 "('Longitude','IntegerType()'",
 "('Location','StringType()'"]